In [ ]:
# Import Functions
import pandas as pd
import os
import time
import csv
import json
import fnmatch
import csv
import sqlalchemy as sql
import numpy as np
test1=None
dbHost='mxnunx41'
dbPort='1521'
dbUser='lims'
dbPassword='Mumwit63'
dbName='hwi1'
dburl='oracle://'

# Default Parameters setting
path='//mxns.loc//shares//NA-Instruments//Prod//Crete//Import//Chemwell//RawFiles//'
archivedDestination='//mxns.loc//shares//NA-Instruments//Prod//Crete//Import//Chemwell//Archived//'
importDest='//mxns.loc//shares//NA-Instruments//Prod//Crete//impf//'
colsOrder=['Sample.Sample_Number', 'Test.Analysis','Result.Name', 'Result.Entry', 'Result.Entered_By']
lab='CRE'
inst='Chemwell'

In [91]:
# get Data from DB
def getDataFromDB(sampleNumbersList):
    dburl = 'oracle://'+dbUser+':'+dbPassword+'@'+dbHost+':'+dbPort+'/'+dbName
    engine=sql.create_engine(dburl)
    resultdf=pd.DataFrame()
    sampleNumbrStr=(', '.join("'" + str(item) + "'" for item in sampleNumbersList))
    queryText="SELECT sample_number, analysis, REPLICATE_COUNT,TEST_NUMBER FROM test"
    queryText=queryText+" WHERE SAMPLE_NUMBER IN ("+sampleNumbrStr+") AND analysis IN ('ALLERGLIAD')"
    conn = engine.connect()
    resultdf = pd.read_sql(queryText, engine)
    conn.close()
    return resultdf

In [ ]:
#write the file
def writeFile(dframe,lab,inst,archDestination,importDest,datapath,fname):
    timestr = time.strftime("%Y%m%d%H%M%S")
    FileName = importDest + lab+'_'+inst+'_' + timestr + '.csv'
    #write file to importer folder
    dframe.to_csv(FileName, index=False, encoding="utf-8",quotechar='"',quoting=csv.QUOTE_NONNUMERIC) # write  out the file to be imported
    destination = archDestination + fname[:-4] +'_'+timestr +'.bak' # Backup the processed file
    os.rename(datapath, destination)
    print("Processed File: "+fname[:-4])
    time.sleep(1)

In [ ]:
# Default Parameters setting
with open('./ConfigFiles/ChemWell.json', 'r') as f:
    config = json.load(f)
print(config)
lab = config['lab']
analysis = config['analysis']
inst = config['inst']
colsOrder=['Sample.Sample_Number', 'Test.Test_Number','Result.Name', 'Result.Entry', 'Result.Entered_By','Result.Instrument']

In [92]:
# Get the List of files
files = os.listdir(pathrawfiles)
# Iterate through List of files
for name in files:
    print(name)

gliadin 190117-1 IT Trial.xls
gliadin 190117-1 IT Trial_raw.xls
gliadin117-1 IT.xls


In [93]:
datapathxl = pathrawfiles + str(name)
dataxl = pd.read_excel(datapathxl)   # read the Excel file from the instrument
dataxl1 = dataxl.drop_duplicates(subset='Sample', keep='first')
dataxl11 = dataxl1[dataxl1['Sample'].map(len) == 11] # Subset only the valid sample numbers
collist = ['Sample', 'Abs', 'Conc']   # define the required columns for the import files
dataxl111 = dataxl11[collist] 			# Subset the required columns for the import files
dataxl111['Sample'] = dataxl111['Sample'].str[2:] # format the sample numbers
dataxl111 = pd.melt(dataxl111, id_vars=['Sample'], value_vars=['Abs', 'Conc']) # Pivot the samples numbers

*** No CODEPAGE record, no encoding_override: will use 'ascii'


c:\users\agundlapalli\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [99]:
# formatting the dataframe to get in respect to the Import Functionality
sampleNumbersList=list(dataxl111['Sample'].unique())
databasedf=getDataFromDB(sampleNumbersList)
databasedf = databasedf.rename(index=str, columns={'sampl_number': 'Sample'})
dataxl111=dataxl111.astype({"Sample": int})
dfResult=pd.merge(dataxl111,databasedf,on='Sample')
dfResult = dfResult.replace('Abs', 'Absorbance')
dfResult = dfResult.replace('Conc', 'Gliadin (Component of Gluten)')
dfResult = dfResult.rename(index=str, columns={'Sample': 'Sample.Sample_Number', 'variable': 'Result.Name','value': 'Result.Entry','test_number':'Test.Test_Number'})
dfResult['Result.Entered_By'] = 'IMPORTER'

In [100]:
dfResult = dfResult.sort_values(by=['Sample.Sample_Number'])
dfResult = dfResult.astype({"Sample.Sample_Number": int})

In [101]:
dfResult = dfResult[colsOrder]
writeFile(dfResult, eachlab, inst, archivedDestination, importDest, datapath, filename)